# LRP Fine tuning
The objective of this notebook is to demonstrate that one can optimise a network post training. 
We demonstrate here that by incorporating attention information, we can improve the quality of models. 
This has several steps: 
- Download the Imagenette dataset (we use this rather than the full imagenet as it is considerably more lightweight)
- Download two pre-trained models on Imagenet (from the VGG family)
- Evaluate the quality of the models on the dataset. 
- Use the better performing model as the "teacher model" 
- Attempt to improve the learner model using the teacher's heatmaps. 
- Periodically evaluate performance
 

In [ ]:
def get_imagenette(path = "./data"):
    """Download the imagenette dataset to data dir"""
    
def get_vgg16():
    """Download a pretrained vgg16 on Imagenet"""
    
def get_vgg19():
    """ Download a pretrained vgg19 on Imagenet"""
    
def 
